# Tutorial Collections Transfer

*Matt Pritchard, June 2024*

Simple example of transfer between Globus tutorial collections.
These are simpler than most because no collection authentication is required.

Further examples here https://globus-sdk-python.readthedocs.io/en/stable/examples/index.html

In [1]:
import os
import datetime as dt
import pytz
import json # so we can pretty-print responses

In [2]:
# Globus SDK imports (requires venv with globus-sdk installed)
from globus_sdk import (
        TransferClient,
        TransferData,
        NativeAppAuthClient, 
        RefreshTokenAuthorizer,
        TransferAPIError
)
from globus_sdk.scopes import TransferScopes
from globus_sdk.tokenstorage import SimpleJSONFileAdapter

In [3]:
# Register your own app at developers.globus.org and use **YOUR OWN** CLIENT_ID here
CLIENT_ID = "3b1925c0-a87b-452b-a492-2c9921d3bd14" # This is the Globus tutorial one: you should register & create your own.

In [4]:
SRC="6c54cade-bde5-45c1-bdea-f4bd71dba2cc" # Globus tutorial collection 1
DST="31ce9ba0-176d-45a5-add3-f37d233ba47d" # Globus tutorial collection 2

In [5]:
AUTH_CLIENT = NativeAppAuthClient(CLIENT_ID)

In [6]:
MY_FILE_ADAPTER = SimpleJSONFileAdapter(
    os.path.expanduser("~/.globus-transfer-tokens.json")
)

In [7]:
def do_login_flow(scopes=TransferScopes.all):
    
    # Do login flow and return tokens
    
    AUTH_CLIENT.oauth2_start_flow(
        requested_scopes=scopes,
        refresh_tokens=True,
    )
    authorize_url = AUTH_CLIENT.oauth2_get_authorize_url()
    print(f"Please go to this URL and login:\n\n{authorize_url}\n")
    auth_code = input("Please enter the code here: ").strip()
    tokens = AUTH_CLIENT.oauth2_exchange_code_for_tokens(auth_code)
    return tokens

In [8]:
# Check for existence of token store file, make it if needed

# Only set to True to force re-login (useful if doesn't work on a subsequent run, but shouldn't be needed)
forceRedoLogin = False

if (not MY_FILE_ADAPTER.file_exists()) or forceRedoLogin:
    # do a login flow, getting back initial tokens
    response = do_login_flow()
    # now store the tokens and pull out the relevant ones for the TransferClient
    MY_FILE_ADAPTER.store(response)
    by_rs = response.by_resource_server
    tokens = by_rs[TransferClient.resource_server]
    print("Generated tokens from login")
else:
    # otherwise, we already did login; load the tokens from that file
    tokens = MY_FILE_ADAPTER.get_token_data(TransferClient.resource_server)
    print("Read tokens from file")
    
print(json.dumps(tokens, indent=4))

Read tokens from file
{
    "scope": "urn:globus:auth:scope:transfer.api.globus.org:all",
    "access_token": "AgllqmE5KJXxqjPmPq3gnz87OkNMExg7yqK6nPWWlWje9XDV7gh8C9GP2vDOXWbkpMm379llvbzEFYxnJDWUOEmd1",
    "refresh_token": "Agy2Y4K9PDBoeDb5m5Yeo603GQylr5E46M8DzlrQ61Yl6OqlkoszUK8X2bYNjvx1xr2wG09wx0g2ljmykqmadD7ByKXWy",
    "token_type": "Bearer",
    "expires_at_seconds": 1721842745,
    "resource_server": "transfer.api.globus.org"
}


In [10]:
# Check the expiry of the token
expiry = dt.datetime.fromtimestamp(tokens["expires_at_seconds"], tz=None)
now = dt.datetime.utcnow()

print("Expiry:\t", expiry)
print("Now:\t", now)

Expiry:	 2024-07-24 17:39:05
Now:	 2024-07-23 14:25:37.294962


In [11]:
# construct the RefreshTokenAuthorizer which writes back to storage on refresh
authorizer = RefreshTokenAuthorizer(
    tokens["refresh_token"],
    AUTH_CLIENT,
    access_token=tokens["access_token"],
    expires_at=tokens["expires_at_seconds"],
    on_refresh=MY_FILE_ADAPTER.on_refresh,
)
# use that authorizer to authorize the activity of the transfer client
transfer_client = TransferClient(authorizer=authorizer)

# initialise a list of required scopes, to check
consent_required_scopes = []

In [12]:
# Now we have a client, we could interact with any collection that 
# doesn't require specific consent. However most do.

# So, try an ls on the source and destination to see if ConsentRequired
# errors are raised

def check_for_consent_required(tc, target):
    try:
        tc.operation_ls(target, path="/")
        print("Consent OK for ",target)
    # catch all errors and discard those other than ConsentRequired
    # e.g. ignore PermissionDenied errors as not relevant
    except TransferAPIError as err:
        if err.info.consent_required:
            consent_required_scopes.extend(err.info.consent_required.required_scopes)

In [13]:
check_for_consent_required(transfer_client, SRC)
check_for_consent_required(transfer_client, DST)

Consent OK for  6c54cade-bde5-45c1-bdea-f4bd71dba2cc
Consent OK for  31ce9ba0-176d-45a5-add3-f37d233ba47d


In [14]:
# the block above may or may not populate the list consent_required_scopes[]
# but if it does, handle ConsentRequired with a new login

if consent_required_scopes:
    print(
        "One of your endpoints requires consent in order to be used.\n"
        "Trying second login to grant consents.\n\n"
    )
    scopes_list = [TransferScopes.all] + consent_required_scopes
    response = do_login_flow(scopes_list)
    MY_FILE_ADAPTER.store(response)
    transfer_client = TransferClient(authorizer=authorizer)

In [15]:
# Create and submit the transfer task.

task_data = TransferData(
    source_endpoint=SRC, destination_endpoint=DST,
    label = "test transfer between tutorial collections"
)
task_data.add_item(
    "/home/share/godata/file2.txt",  # source
    "/~/file2.txt",  # dest
)


def do_submit(client):
    task_doc = client.submit_transfer(task_data)
    task_id = task_doc["task_id"]
    print(f"submitted transfer, task_id={task_id}")

In [16]:
try:
    do_submit(transfer_client)
except TransferAPIError as err:
    if err.info.consent_required:
        print(
            "Consent error: one or more collection requires consent, please login again."
        )
    elif (err.info.authorization_parameters and err.info.authorization_parameters.session_required_single_domain):
        print(err.info.authorization_parameters.session_message," : ",err.info.authorization_parameters.session_required_single_domain)
        print(
            f"Your authentication with domain {err.info.authorization_parameters.session_required_single_domain} needs to be refreshed. Please go to https://app.globus.org and navigate to the collection manually to re-authenticate, then retry here."
        )
    else:
        print(
            "A transfer API error occurred\n",
            err
        )


submitted transfer, task_id=96659e72-48ff-11ef-8dfd-19f3c8361d4f
